# 통합 식품영양성분DB 정리
### - [클리닝](#1.-클리닝)
### - [타입 변경(object -> float)](#2.-타입-변경)
### - [데이터 단위 통일(영양소 g으로 통일)](#3.-영양소-별-데이터-단위-통일)
### - [각 식품의 영양소 최대 값 행 추가](#4.-)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re


In [2]:
food_nutrients = pd.read_excel('통합 식품영양성분DB_20211106.xlsx', skiprows=[0,1,2])
food_nutrients.head()

FileNotFoundError: [Errno 2] No such file or directory: '통합 식품영양성분DB_20211106.xlsx'

In [ ]:
pd.options.display.max_columns = 241
pd.set_option('display.max.colwidth', 50)

food_nutrients.info()

## 1. 클리닝 

#### 필요없는 colum 제거 및 float로 변환하기 위해 데이터 정제

In [ ]:
food_nutrients = food_nutrients.drop(['SAMPLE_ID', '식품코드', '연도', '지역 / 제조사',\
                                      '채취시기', '폐기율(%)', '성분표출처', '발행기관'], axis = 1)

In [ ]:
food_nutrients.head()

In [ ]:
food_nutrients_copy = food_nutrients.copy()
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = food_nutrients.loc[: ,'단백질(g)':'카페인(㎎)'].replace(['-', 'tr', 'Tr'], 0)

In [ ]:
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'미만'), 0)
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'이하'), 0)
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = \
        food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].replace(re.compile(r'<'), 0)

food_nutrients_copy.head()

## 2. 타입 변경

In [ ]:
food_nutrients_copy = food_nutrients_copy.astype({'1회제공량':'float'})
food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] = food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'].astype(float)

food_nutrients_copy.head()

## 3. 영양소 별 데이터 단위 통일

In [ ]:
gTomg = 1000

food_nutrients_copy.loc[:, [col for col in food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] if col.find('mg')]] / gTomg
food_nutrients_copy.loc[:, [col for col in food_nutrients_copy.loc[: ,'단백질(g)':'카페인(㎎)'] if col.find('㎍')]] / (gTomg * gTomg)

##### 영향소별 비율을 구하기 위해 데이터를 살펴본 결과 데이터가 복잡.?
ex)아라키돈산, 에이코사펜타에노산 같은 영양소 데이터의 경우 g, mg, %로 나뉘어져 있으며 식품에 따라 표기된 위치가 다르다 다행히 g에도 적혀있고 mg에도 적히는 등 중복으로 된 것은 없어서 통합하면 될 것 같다. % 에 표기된 경우 연산 없이 바로 %값을 사용 -> %값이 있는 데이터의 경우 연산을 진행할 때 제외하고 한다....
